In [3]:
library(DiceKriging)
source("sobolGP.R")
source("sobolGPmethods.R")
source("pickfreeze.R")

#--------------------------------------#
# kriging model building
#--------------------------------------#

d <- 2; nobs <- 30
design.fact <- expand.grid(x1=seq(0,1,length=nobs), x2=seq(0,1,length=nobs))
y <- apply(design.fact, 1, branin) 

# m <- km(design=design.fact, response=y)
m <- km(design=design.fact, response=y, nugget = 1e-1 * var(y), covtype = "gauss")

#--------------------------------------#
# sobol samples & candidate points
#--------------------------------------#

n <- 1000
X1 <- data.frame(matrix(runif(d * n), nrow = n))
X2 <- data.frame(matrix(runif(d * n), nrow = n))

candidate <- data.frame(matrix(runif(d * 100), nrow = 100))

#--------------------------------------#
# Kriging-based Sobol
#--------------------------------------#

res <- sobolGP(
model = m,
type="UK",
MCmethod="sobol",
X1,
X2,
nsim = 5,
conf = 0.95,
nboot=1,
sequential = FALSE,
candidate,
sequential.tot=FALSE,
max_iter = 1000
) 

res


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  gauss 
  - nugget : 302.4066 
  - parameters lower bounds :  1e-10 1e-10 
  - parameters upper bounds :  2 2 
  - variance bounds :  272.1659 39049.73 
  - best initial criterion value(s) :  -3543.031 

N = 3, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=         3543  |proj g|=        1.863
At iterate     1  f =         3484  |proj g|=        1.8267
At iterate     2  f =       3479.4  |proj g|=        1.7876
At iterate     3  f =       3478.9  |proj g|=        1.5029
At iterate     4  f =       3477.7  |proj g|=        1.4309
At iterate     5  f =       3477.4  |proj g|=        1.3799
At iterate     6  f =       3477.4  |proj g|=       0.21583
At iterate     7  f =       3477.4  |proj g|=       0.19015
At iterate     8  f =       3477.4  |proj g|=    


Method: sobol

Model runs: 900 

Number of GP realizations: 5 

Kriging type: UK 

    estimate   std. error min. c.i. max. c.i.
S1 0.1665604 0.0003987598 0.1659992 0.1669855
S2 0.2741401 0.0009210800 0.2728590 0.2749817


In [ ]:
 estimate  std. error min. c.i. max. c.i.
S1 0.1187959 0.015753768 0.1050829 0.1416353
S2 0.2321796 0.009414102 0.2189813 0.2425176

In [ ]:
 estimate  std. error min. c.i. max. c.i.
S1 0.1331096 0.004986319 0.1265005 0.1384144
S2 0.2320974 0.007212113 0.2254677 0.2406317

In [ ]:
estimate  std. error min. c.i. max. c.i.
S1 0.1390570 0.001771327 0.1369693 0.1412450
S2 0.3312613 0.001234814 0.3293628 0.3320793

In [295]:
library(DiceKriging)
source("sobolGP.R")
source("sobolGPmethods.R")
source("pickfreeze.R")
source("sobolGP_th.r")
source("pickfreeze_th.R")

In [91]:
#--------------------------------------#
# Data
#--------------------------------------#
d <- 2; n <- 39
dat <- read.table('C:/Users/berkc/git/Physics-Informed-Machine-Learning/data/labeled_data.dat')
x_exp <- dat[,1:2]
y_exp <- dat[,ncol(dat)-1]
colnames(x_exp)<- c("X1","X2")
# x_exp <- data.frame("T" = c(210.00000000000000,260.00000000000000,255.00000000000000,234.00000000000000,
#                         223.00000000000000,242.00000000000000,219.00000000000000,
#                         239.00000000000000,223.00000000000000,239.00000000000000,247.00000000000000,
#                         240.00000000000000,255.00000000000000,239.00000000000000,252.00000000000000,
#                         240.00000000000000,247.00000000000000,240.00000000000000,239.00000000000000,
#                         239.00000000000000,259.30000000000000,259.30000000000000,210.00000000000000,
#                         234.00000000000000,260.00000000000000,226.00000000000000,240.00000000000000,
#                         234.00000000000000,219.00000000000000,242.00000000000000,260.00000000000000,
#                         223.00000000000000,247.00000000000000,255.00000000000000,252.00000000000000,
#                         226.00000000000000,250.00000000000000,259.30000000000000,259.00000000000000), 
#                           "S" = c(28.00000000000000,35.00000000000000,46.00000000000000,34.00000000000000,29.00000000000000,
#                                   26.00000000000000,23.00000000000000,25.20000000000000,29.00000000000000,
#                                   36.60000000000000,37.00000000000000,42.00000000000000,46.00000000000000,
#                                   32.80000000000000,21.00000000000000,42.00000000000000,37.00000000000000,
#                                   42.00000000000000,29.00000000000000,40.40000000000000,37.20000000000000,
#                                   37.20000000000000,28.00000000000000,34.00000000000000,35.00000000000000,
#                                   18.60000000000000,15.00000000000000,34.00000000000000,23.00000000000000,
#                                   26.00000000000000,35.00000000000000,29.00000000000000,37.00000000000000,
#                                   46.00000000000000,21.00000000000000,18.60000000000000,15.00000000000000,
#                                   37.20000000000000,43.00000000000000))
# y_exp <- c(0.05035167,0.01216063,0.0447357,0.03123432,0.02438436,0.0074955,0.01354554,0.00824909,0.02870565,0.04645434,0.02359506,
#        0.05660703,0.04627013,0.02638973,0.00632564,0.04789699,0.01626827,0.07735481,0.01441945,
#        0.05652293,0.01666471,0.01358868,0.05432798,0.03850274,0.01010751,0.01129286,0.009357,0.03579652,0.01614062,0.00770211,0.0105933,0.03315482,0.01352198,0.05269549,0.00490424,
#        0.00895544,0.01003564,0.01345683,0.02496436)


normalise <- function(y) {
    (y - min(y)) / (max(y)-min(y))
}

normalize <- function(x, na.rm = TRUE) {
    ranx <- range(x, na.rm = na.rm)
    (x - ranx[1]) / diff(ranx)
}
x_exp.norm = apply(x_exp, 2, normalize)
y_exp.norm = normalise(y_exp)

In [271]:
#--------------------------------------#
# kriging model building
#--------------------------------------#
# m <- km(design=x_exp.norm, response=y_exp.norm, nugget = 1e-3 * var(y_exp.norm))
m <- km(design=x_exp.norm, response=y_exp.norm, nugget = 1e-1 * var(y_exp.norm), covtype = "gauss")


optimisation start
------------------
* estimation method   : MLE 
* optimisation method : BFGS 
* analytical gradient : used
* trend model : ~1
* covariance model : 
  - type :  gauss 
  - nugget : 0.006616911 
  - parameters lower bounds :  1e-10 1e-10 
  - parameters upper bounds :  2 2 
  - variance bounds :  0.00595522 0.7046606 
  - best initial criterion value(s) :  21.48118 

N = 3, M = 5 machine precision = 2.22045e-16
At X0, 0 variables are exactly at the bounds
At iterate     0  f=      -21.481  |proj g|=       1.8093
At iterate     1  f =      -25.325  |proj g|=       0.64326
At iterate     2  f =      -25.826  |proj g|=       0.48611
At iterate     3  f =      -25.832  |proj g|=       0.33871
At iterate     4  f =      -25.865  |proj g|=        1.3784
At iterate     5  f =      -25.888  |proj g|=       0.89089
At iterate     6  f =      -25.891  |proj g|=       0.49831
At iterate     7  f =      -25.893  |proj g|=      0.089584
At iterate     8  f =      -25.893  |proj g|

In [293]:
#--------------------------------------#
# sobol samples & candidate points
#--------------------------------------#
n <- 5000
X1 <- data.frame(matrix(runif(d * n), nrow = n))
X2 <- data.frame(matrix(runif(d * n), nrow = n))

candidate <- data.frame(matrix(runif(d * 10), nrow = 10))

In [263]:
# pp <- predict(m, X1, "UK")
# pp$sd^2
# pp$m

sp <- simulate(m, X1, nsim=10)
apply(sp,1,mean)
apply(sp,1,var)

[1] 0.3456427 0.2917140 0.2483196 0.2337335 0.2352205 0.3238606 0.3566461
 [8] 0.2783107 0.1992333 0.5564642

[1] 0.09942861 0.04907417 0.05325340 0.04086598 0.03758951 0.05611879
 [7] 0.05314970 0.12167334 0.07836172 0.01658300

In [275]:
#--------------------------------------#
# Kriging-based Sobol
#--------------------------------------#
res <- sobolGP_th(
model = m,
type="UK",
MCmethod="sobol",
X1,
X2,
nsim = 1,
conf = 0.95,
nboot=1,
sequential = FALSE,
candidate,
sequential.tot=FALSE,
max_iter = 1000
) 

res

$S1
[1] 0.2265595

$S2
[1] 0.5574962




Method: sobol

Model runs: 39 

Number of GP realizations: 1 

Kriging type: UK 

    estimate std. error min. c.i. max. c.i.
S1 0.2265595         NA 0.2265595 0.2265595
S2 0.5574962         NA 0.5574962 0.5574962


In [296]:
#--------------------------------------#
# Kriging-based Sobol
#--------------------------------------#
res <- sobolGP(
model = m,
type="UK",
MCmethod="sobol",
X1,
X2,
nsim = 30,
conf = 0.95,
nboot=1,
sequential = FALSE,
candidate,
sequential.tot=FALSE,
max_iter = 1000
) 

res
# plot(res)
# x <- ask(res)
# y <- branin(x)
# The following line doesn't work (uncorrected bug: 
#     unused argument in km(), passed by update(), eval(), tell.sobolGP() ??)
#res.new <- tell(res,y,x)
#res.new



Method: sobol

Model runs: 39 

Number of GP realizations: 30 

Kriging type: UK 

    estimate std. error min. c.i. max. c.i.
S1 0.2484554 0.03813858 0.1729196 0.2984901
S2 0.6335153 0.02034616 0.5978824 0.6760371


In [ ]:
Method: sobol

Model runs: 39 

Number of GP realizations: 30 

Kriging type: UK 

    estimate std. error min. c.i. max. c.i.
S1 0.2514368 0.04645888 0.1603408 0.3180157
S2 0.5911329 0.03156081 0.5379306 0.6510354

THEIR FUNC.
Method: sobol

Model runs: 39 

Number of GP realizations: 30 

Kriging type: UK 

    estimate std. error min. c.i. max. c.i.
S1 0.2484554 0.03813858 0.1729196 0.2984901
S2 0.6335153 0.02034616 0.5978824 0.6760371

In [139]:
# # mm <- res[[3]]$cov
# # nrow(mm)
# res2 <- res[[2]]
# nrow(res2)
# ncol(res2)

# res3 <- res[[3]]$cov
# nrow(res3)
# ncol(res3)

# res3 <- res[[3]]$mean2
# length(res3)

[1] 10

[1] 210

[1] 210

[1] 10

[1] 10

In [116]:
# sqrt(res[[1]]$S$var)

ERROR: Error in sqrt(res[[1]]$S$var): non-numeric argument to mathematical function


In [115]:
# sqrt(res[[1]]$S$varPG)

ERROR: Error in sqrt(res[[1]]$S$varPG): non-numeric argument to mathematical function


In [114]:
# output$T$mean

ERROR: Error in eval(expr, envir, enclos): object 'output' not found


In [174]:
# names(res)

[1] "call" "S"

In [117]:
# res[[1]]$S[[1]]

NULL